## purpose: wifiの特徴量の見つかって、予測をする。この予測値は後処理に使う予定。

In [4]:
import sys
sys.path.append("..")

import os
import json
import glob
import cv2
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import seaborn as sns

from PIL import Image, ImageOps
from skimage import io
from skimage.color import rgba2rgb, rgb2xyz
from tqdm import tqdm
from dataclasses import dataclass
from math import floor, ceil

from main import calibrate_magnetic_wifi_ibeacon_to_position
from main import extract_magnetic_strength
from io_f import read_data_file
from visualize_f import visualize_trajectory, visualize_heatmap
from main import calibrate_magnetic_wifi_ibeacon_to_position
from main import extract_magnetic_strength
from main import extract_wifi_rssi, extract_wifi_count
import processing
import os

import lightgbm as lgb
sns.set_theme()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
feature_dir = "../resource/wifi_features"
# the metric used in this competition
def comp_metric(xhat, yhat, fhat, x, y, f):
    intermediate = np.sqrt(np.power(xhat - x,2) + np.power(yhat-y,2)) + 15 * np.abs(fhat-f)
    return intermediate.sum()/xhat.shape[0]

# get our train and test files
train_files = sorted(glob.glob(os.path.join(feature_dir, 'train/*_train.csv')))
test_files = sorted(glob.glob(os.path.join(feature_dir, 'test/*_test.csv')))
ssubm = pd.read_csv('../sample_submission.csv', index_col=0)

In [5]:
predictions = list()

for e, file in enumerate(train_files):
    data = pd.read_csv(file, index_col=0)
    test_data = pd.read_csv(test_files[e], index_col=0)

    # simple grid search for tuning using path and groupkfold cv
    #for i in [127]:
    #    for n in [75]:
    #        modely = lgb.LGBMRegressor(
    #        n_estimators=n, num_leaves=i, n_jobs=1)
    #        gkf = model_selection.GroupKFold()
    #        scores = model_selection.cross_val_score(
    #            modely, x_train, y_trainy, scoring='neg_mean_squared_error',
    #            cv=gkf, groups=data.iloc[:,-1], n_jobs=5)
    #        print(i, n, l, scores, scores.mean())
    

    x_train = data.iloc[:,:-4]
    y_trainy = data.iloc[:,-3]
    y_trainx = data.iloc[:,-4]
    y_trainf = data.iloc[:,-2]

    modely = lgb.LGBMRegressor(
        n_estimators=125, num_leaves=90)
    modely.fit(x_train, y_trainy)

    modelx = lgb.LGBMRegressor(
        n_estimators=125, num_leaves=90)
    modelx.fit(x_train, y_trainx)

    modelf = lgb.LGBMClassifier(
        n_estimators=125, num_leaves=90)
    modelf.fit(x_train, y_trainf)
    
    test_predsx = modelx.predict(test_data.iloc[:,:-1])
    test_predsy = modely.predict(test_data.iloc[:,:-1])
    test_predsf = modelf.predict(test_data.iloc[:,:-1])
    
    test_preds = pd.DataFrame(np.stack((test_predsf, test_predsx, test_predsy))).T
    test_preds.columns = ssubm.columns
    test_preds.index = test_data["site_path_timestamp"]
    test_preds["floor"] = test_preds["floor"].astype(int)
    predictions.append(test_preds)
# generate prediction file 
all_preds = pd.concat(predictions)
all_preds = all_preds.reindex(ssubm.index)
all_preds.to_csv('../output/wifi_prediction.csv')

In [10]:
# count_path = "0"
# tss = []
# for v in test_data["site_path_timestamp"]:
#     data_ppath = v.split("_")
#     if count_path == "0":
#         count_path = data_ppath[1]
#     if count_path != data_ppath[1]:
#         break
#     tss.append(data_ppath[-1])
# tss

['0000000000008',
 '0000000003114',
 '0000000005522',
 '0000000008223',
 '0000000013143',
 '0000000019651',
 '0000000024179',
 '0000000032725',
 '0000000040244',
 '0000000043815',
 '0000000052600',
 '0000000060237',
 '0000000062986',
 '0000000064849',
 '0000000067224',
 '0000000076427',
 '0000000079143',
 '0000000080918',
 '0000000082830',
 '0000000093080',
 '0000000096327',
 '0000000102271',
 '0000000110504']